In [140]:
import matplotlib.pyplot as plt
import matplotlib.image as mping 
import numpy as np
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import cv2

#Steps:            Parameters:
#1:Get image       (nothing)         
#2:Get Masked      (Thresholds)             
#3:Darken          (level of change) 
#4:Merge all       (nothing)         
#5:Convert to grey (nothing)  
#6:Blur            (kernel_size)
#7:Canny edge      (LowT,HighT)
#8:Area            (shape, size)
#9:Hough lines     (Threshold, min line length, max line gap, rho, theta)
#10:Draw           (Lines to ignore)

In [141]:
def ColorSelect(image, low_rgb, high_rgb):
    mask = np.copy(image)
    thresholds = (image[:,:,0] > high_rgb[0]) \
                   | (image[:,:,0] < low_rgb[0]) \
                   | (image[:,:,1] > high_rgb[1]) \
                   | (image[:,:,1] < low_rgb[1]) \
                   | (image[:,:,2] > high_rgb[2]) \
                   | (image[:,:,2] < low_rgb[2])   
                
    mask[thresholds] = [0,0,0]
    return mask

In [142]:
def RegionOfInterest(original, points):
    
    image = np.copy(original)
    mask = np.zeros_like(original)   
    ignore_mask_color = 255
    
    cv2.fillPoly(mask, points, ignore_mask_color)
    
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

In [145]:
def LaneLineDetector(original_image):
    #1
    image=np.copy(original_image) 
    
    #2
    yellow_low_thresholds = [180,160,0]
    yellow_high_thresholds = [255,255,140] 
    white_low_thresholds = [190,190,190]
    white_high_thresholds = [255,255,255]
                             
    white_mask = ColorSelect(image,white_low_thresholds,white_high_thresholds)
    yellow_mask = ColorSelect(image,yellow_low_thresholds,yellow_high_thresholds)

    #3 
    change = 100
    darker_image = np.where(image<change,0,image-change)

    #4                             
    whited_image = cv2.bitwise_or(darker_image, white_mask)
    masked_image = cv2.bitwise_or(whited_image, yellow_mask)
   
    #5                        
    grey_image = cv2.cvtColor(masked_image, cv2.COLOR_RGB2GRAY)
    
    #6
    kernel_size = 7                  
    blurred_image = cv2.GaussianBlur(grey_image,(kernel_size, kernel_size), 0)
    
    #7
    low_threshold = 50
    high_threshold = 150    
    edged_image = cv2.Canny(blurred_image, low_threshold, high_threshold)
 
    #8
    p1,p2,p3,p4  = [190,665],[600,430],[700,430],[1110,665]
    points = np.array([[p1,p2,p3,p4]], dtype=np.int32) 
    region_image= RegionOfInterest(edged_image, points)

    #9
    rho = 1
    theta = np.pi/180
    threshold = 20
    min_line_length = 50
    max_line_gap = 60
    lined_image = np.zeros_like(image)

    lines = cv2.HoughLinesP(region_image, rho, theta, threshold, np.array([]),
                                             min_line_length, max_line_gap)
    #10
    line_width = 5

    for line in lines:
        for x1,y1,x2,y2 in line: #No lines with little slope. No lines that cross the middle.
            if ((np.absolute(y1-y2)>(np.absolute(x1-x2)*.5)) and ((x1>690 and x2>690) or (680>x1 and 680>x2))):
                cv2.line(lined_image,(x1,y1),(x2,y2),(255,0,0),line_width)

    laned_image = cv2.addWeighted(image, 0.8, lined_image, 1, 0) 

    return laned_image 

In [139]:
final_clip_output = 'final_output.mp4'
test_clip = VideoFileClip("test.mp4")
final_clip = test_clip.fl_image(LaneLineDetector)
%time final_clip.write_videofile(final_clip_output, audio=False)

[MoviePy] >>>> Building video final_output.mp4
[MoviePy] Writing video final_output.mp4



 62%|██████▏   | 155/251 [00:30<00:17,  5.57it/s]


100%|██████████| 251/251 [00:49<00:00,  5.31it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: final_output.mp4 

CPU times: user 1min 57s, sys: 9.28 s, total: 2min 7s
Wall time: 51.1 s


In [144]:
HTML("""
<video width="640" height="300" controls>
  <source src="{0}" type="video/mp4">
</video>
""".format(final_clip_output))